In [ ]:
import numpy as np
from qunetsim.components import Host
from qunetsim.components import Network
from qunetsim.objects import Logger

Logger.DISABLED = True
def protocol_1(host, receiver, designated_node):
    print('Sending EPR pair') #generate and send an epr pair
    epr_id, ack_arrived = host.send_epr(receiver, await_ack=True)

    if ack_arrived:
        # measure the epr pair
        q = host.get_epr(receiver, q_id=epr_id)
        q_measurement = q.measure()
        print(f'Host 1 measured: {q_measurement}')    
        # Send measurement result to bob through a classical channel
        host.send_classical(designated_node, str(q_measurement))
        print(f'Host 1 sent measurement result to {designated_node}')
    else:
        print('The EPR pair was not properly established')
    print('Sender protocol done')

def protocol_2(host, sender, designated_node):
    q = host.get_epr(sender, wait=5)
    if q is not None:
        host.x_error_rate=.5
        if np.random.rand() < host.x_error_rate:
            q.X()
        q_measurement=q.measure()
        print(f'Host 2 measured: {q_measurement}')
        #implement classical channel
        host.send_classical(designated_node, str(q_measurement))
        print(f'Host 2 sent measurement result to {designated_node}')
    else:
        print('Host 2 did not receive an EPR pair')
print('Receiver protocol done')

def designated_protocol(host, node_1, node_2):
    #get the classical messages
    result1 = host.get_classical(node_1, wait=10)
    result2 = host.get_classical(node_2, wait=10)
    #get the state of the qubit. the results are a list, so need to extract the content and turn it into integers
    result1_content=int(result1[0].content)
    result2_content=int(result2[0].content)
    #compute parity using a XOR gate 
    parity = result1_content ^ result2_content
    print(f'Computed parity at designated node: {parity}')
    if parity == 0:
        print("Parity check passed. No errors detected in logical bits.")
    else:
        print("Parity check failed. Errors might have occurred.")

def main():
    #set up the network
    network = Network.get_instance()
    nodes = ['Alice', 'Bob', 'Eve']
    network.start(nodes)

    host_Alice = Host('Alice')
    host_Alice.add_connection('Bob')
    host_Alice.start()

    host_Bob = Host('Bob')
    host_Bob.add_connection('Alice')
    host_Bob.add_connection('Eve')
    host_Bob.start()

    host_Eve = Host('Eve')
    host_Eve.add_connection('Bob')
    host_Eve.start()

    network.add_host(host_Alice)
    network.add_host(host_Bob)
    network.add_host(host_Eve)

    #Alice and Eve are hosts, Bob is a designated node 
    host_Alice.run_protocol(protocol_1, (host_Eve.host_id, host_Bob.host_id))
    host_Eve.run_protocol(protocol_2, (host_Alice.host_id, host_Bob.host_id))
    host_Bob.run_protocol(designated_protocol, ('Eve', 'Alice'))  
    
if __name__ == '__main__':
    main()

Receiver protocol done
Sending EPR pair


Host 2 measured: 0
Host 2 sent measurement result to Bob
Host 1 measured: 1
Host 1 sent measurement result to Bob
Sender protocol done
Computed parity at designated node: 1
Parity check failed. Errors might have occurred.
